# Process

## 0. Dependency Parsing

In [143]:
import spacy
nlp=spacy.load('en_core_web_sm/en_core_web_sm-3.4.1/')

text='John hit the ball'

list_text = []
for index,token in enumerate(nlp(text)):
    print(index, token.text,'=>',token.dep_,'=>',token.head.text)
    list_text.append((index,token.text,token.dep_)) 

0 John => nsubj => hit
1 hit => ROOT => hit
2 the => det => ball
3 ball => dobj => hit


In [144]:
list_text

[(0, 'John', 'nsubj'),
 (1, 'hit', 'ROOT'),
 (2, 'the', 'det'),
 (3, 'ball', 'dobj')]

## 1. Stack : Enqueue and Dequeue 

In [145]:
from collections import deque
class Stack(object):
    def __init__(self):
        self.container = deque()
    def push(self,value):
        self.container.append(value)
    def pop(self):
        if len(self.container) != 0:
            return self.container.pop()  
        else: raise IndexError("An empty deque")
    # def peek(self):
    #     return  self.container[-1]
    # def is_empty(self):
    #     return len(self.container)==0
    def size(self):
        return len(self.container)

In [146]:
s = Stack()
#push items
for i in range(len(list_text)):
    s.push(list_text[i][1])
    
print("Initial queue")
print(s.container)

#pop items
for i in range(len(list_text)):
    s.pop()
    
print("\nQueue after removing elements")
print(s.container)

Initial queue
deque(['John', 'hit', 'the', 'ball'])

Queue after removing elements
deque([])


## 2. Transition-based : Stack and Buffer

In [147]:
class Configuration(object):
    def __init__(self,depency_list):
        self.stack = list()
        self.buffer = depency_list
        self.arcs = list()
    def __str__(self):
        return f'Stack : {self.stack} \nBuffer : {self.buffer} \nArcs : {self.arcs}'

class Transition(object):
    def __init__(self):
        pass

    def left_arc(self,config,relation): 
        if (len(config.buffer) <= 0) or (len(config.stack) <= 0):
            return -1
        if config.buffer[0] == 0:
            return -1
        #pop top of stack -> append arc relation
        index_i = config.stack[len(config.stack)-1]
        index_j = config.buffer[0]
        config.stack.pop()
        config.arcs.append((index_j, relation, index_i)) #buffer to stack

    def right_arc(self,config,relation):
        if (len(config.buffer) <= 0) or (len(config.stack) <= 0):
            return -1
        #pop top of buffer -> append arc relation
        index_i = config.stack[len(config.stack)-1]
        index_j = config.buffer.pop(0)
        # config.buffer.pop(0)
        # config.stack.append(index_j)
        config.arcs.append((index_i, relation, index_j)) #stack to buffer

    def shift(self,config): 
        if len(config.buffer) <= 0:
            return -1
        #move buffer to stack
        index_i = config.buffer.pop(0)
        config.stack.append(index_i)


In [151]:
new_list = list_text.copy()
new_list

[(0, 'John', 'nsubj'),
 (1, 'hit', 'ROOT'),
 (2, 'the', 'det'),
 (3, 'ball', 'dobj')]

In [152]:
config = Configuration(new_list)
print(config)

Stack : [] 
Buffer : [(0, 'John', 'nsubj'), (1, 'hit', 'ROOT'), (2, 'the', 'det'), (3, 'ball', 'dobj')] 
Arcs : []


In [153]:
Tran = Transition()
Tran.shift(config)
Tran.left_arc(config,config.buffer[0][2])
Tran.shift(config)
Tran.shift(config)
Tran.left_arc(config,config.buffer[0][2])
Tran.right_arc(config,config.buffer[0][2])
print(config)

Stack : [(1, 'hit', 'ROOT')] 
Buffer : [] 
Arcs : [((1, 'hit', 'ROOT'), 'ROOT', (0, 'John', 'nsubj')), ((3, 'ball', 'dobj'), 'dobj', (2, 'the', 'det')), ((1, 'hit', 'ROOT'), 'dobj', (3, 'ball', 'dobj'))]


### Reference : Transition-Based
- https://www.youtube.com/watch?v=xG0MXIPVUdk&list=PLyyEwPZh6aHpDieqWwr7hTgreKnYSY9Ox&index=60
- https://www.youtube.com/watch?v=2jLk93iIyrw&list=PLr9TFf9GjancAHiiP5cqGmYsZad99Uf_h&index=39
- https://www.youtube.com/watch?v=f5-hTA9hA3s&list=WL&index=8
- https://www.youtube.com/watch?v=oLHnqGmQtI4

## 3. Chu-Liu-Edmonds : Maximum Spaning Tree

In [88]:
class Graph_Matrix: 
    adj = []
    def __init__(self, v, e): 
        self.v = v
        self.e = e
        Graph_Matrix.adj = [[0 for i in range(self.v)] for j in range(self.v)]
        
    def addEdge(self, s, e): #Add to Matrix
        Graph_Matrix.adj[s-1][e-1] = Graph_Matrix.adj[e-1][s-1] = 1


In [91]:
size = len(list_text)
Matrix = Graph_Matrix(size,size)
Matrix.addEdge(1, 2)
Matrix.addEdge(2, 6)
Matrix.addEdge(3, 4)
Matrix.addEdge(4, 6)
Matrix.addEdge(5, 3)
Matrix.addEdge(5, 4)
Matrix.adj

[[0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 1, 0],
 [0, 0, 1, 0, 1, 1],
 [0, 0, 1, 1, 0, 0],
 [0, 1, 0, 1, 0, 0]]

## 4. MST

## 4. Convert to Binary Tree

In [5]:
class Node(object):
    def __init__(self, key):
        self.left  = None
        self.right = None
        self.key = key
        
    def insert(self, key):
        if(self.key):
            if(key < self.key):
                if(self.left == None):
                    self.left = Node(key)
                else:
                    self.left.insert(key)
            
            elif(key >= self.key):
                if(self.right == None):
                    self.right = Node(key)
                else:
                    self.right.insert(key)
        else:
            self.key = key
    
    def printT(self):        
        if (self.left):
            self.left.printT()
        print(self.key)
        if (self.right):
            self.right.printT()
    
    def delete(self,value):
        if value < self.key: #CaseD
            if self.left:
                self.left = self.left.delete(value)
        elif value > self.key: #CaseC
            if self.right:
                self.right = self.right.delete(value)
        else:
            if self.left == None and self.right == None: #Case A and B
                return None
            elif self.left == None:
                return self.right
            elif self.right == None: 
                return self.left

In [13]:
root = Node(len(list_text))
for i in list_text:
    root.insert(i[0])

root.printT()

1
2
3
4
5
6


In [ ]:
def 